In [1]:
from dask.distributed import LocalCluster,Client
import xarray as xr
import dask.array as da
import glob
import rasterio
import numpy as np
import pandas as pd
import pingouin as pg

In [2]:
# Specify the desired number of cores and threads per worker
n_cores = 32  
threads_per_worker = 2

# Specify a custom port for the Dask dashboard
custom_dashboard_port = 8999

In [3]:
# Create a LocalCluster with the specified configuration
cluster = LocalCluster(n_workers=n_cores // threads_per_worker, threads_per_worker=threads_per_worker,dashboard_address=f":{custom_dashboard_port}")
#This means 16 physical cores will be used and each core will do parallelisation job. 

In [4]:
# Create a Dask client to connect to the cluster
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8999/status,
Dashboard: http://127.0.0.1:8999/status,Workers: 16
Total threads: 32,Total memory: 503.54 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45735,Workers: 16
Dashboard: http://127.0.0.1:8999/status,Total threads: 32
Started: Just now,Total memory: 503.54 GiB
Comm: tcp://127.0.0.1:44465,Total threads: 2
Dashboard: http://127.0.0.1:41937/status,Memory: 31.47 GiB
Nanny: tcp://127.0.0.1:35239,


In [40]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:45735")  # Replace with your scheduler address
print(client.get_versions(check=True))

{'scheduler': {'host': {'python': '3.11.5.final.0', 'python-bits': 64, 'OS': 'Linux', 'OS-release': '5.4.0-164-generic', 'machine': 'x86_64', 'processor': 'x86_64', 'byteorder': 'little', 'LC_ALL': 'None', 'LANG': 'en_US'}, 'packages': {'python': '3.11.5.final.0', 'dask': '2023.11.0', 'distributed': '2023.11.0', 'msgpack': '1.0.3', 'cloudpickle': '2.2.1', 'tornado': '6.3.3', 'toolz': '0.12.0', 'numpy': '1.26.2', 'pandas': '2.1.1', 'lz4': '4.3.2'}}, 'workers': {'tcp://127.0.0.1:33269': {'host': {'python': '3.11.5.final.0', 'python-bits': 64, 'OS': 'Linux', 'OS-release': '5.4.0-164-generic', 'machine': 'x86_64', 'processor': 'x86_64', 'byteorder': 'little', 'LC_ALL': 'None', 'LANG': 'en_US'}, 'packages': {'python': '3.11.5.final.0', 'dask': '2023.11.0', 'distributed': '2023.11.0', 'msgpack': '1.0.3', 'cloudpickle': '2.2.1', 'tornado': '6.3.3', 'toolz': '0.12.0', 'numpy': '1.26.2', 'pandas': '2.1.1', 'lz4': '4.3.2'}}, 'tcp://127.0.0.1:35491': {'host': {'python': '3.11.5.final.0', 'python-

In [5]:
#Relace 'your/folder/path' with the path to your folder containing TIF files
folder_path = '/home/khanalp/master/Master'

# Use the glob function to get a list of TIF files
tif_files = glob.glob(f"{folder_path}/*.tif")

In [6]:
#I grab all the variables

NIRv = tif_files[0]
SSM = tif_files[2]
TWS = tif_files[4]
Ta = tif_files[3]
Rn = tif_files[1]

In [7]:
# Open the GeoTIFF file as an xarray.Dataset
NIRv = xr.open_dataset(NIRv)
SSM =  xr.open_dataset(SSM)
TWS = xr.open_dataset(TWS)
Ta = xr.open_dataset(Ta)
Rn = xr.open_dataset(Rn)

In [8]:
# Rename dimensions
NIRv = NIRv.rename({'x': 'longitude', 'y': 'latitude', 'band': 'time','band_data': 'NIRv'})
SSM = SSM.rename({'x': 'longitude', 'y': 'latitude', 'band': 'time','band_data': 'SSM'})
TWS = TWS.rename({'x': 'longitude', 'y': 'latitude', 'band': 'time','band_data': 'TWS'})
Ta = Ta.rename({'x': 'longitude', 'y': 'latitude', 'band': 'time','band_data': 'Ta'})
Rn = Rn.rename({'x': 'longitude', 'y': 'latitude', 'band': 'time','band_data': 'Rn'})

In [9]:
NIRv = NIRv.isel(
    latitude=slice(0, 1248),  # (0., 125.) deg
    longitude=slice(0, 1248),  # (90., -35.) deg
)
SSM = SSM.isel(
    latitude=slice(0, 1248),  # (0., 125.) deg
    longitude=slice(0, 1248),  # (90., -35.) deg
)
TWS = TWS.isel(
    latitude=slice(0, 1248),  # (0., 125.) deg
    longitude=slice(0, 1248),  # (90., -35.) deg
)
Ta = Ta.isel(
    latitude=slice(0, 1248),  # (0., 125.) deg
    longitude=slice(0, 1248),  # (90., -35.) deg
)
Rn = Rn.isel(
    latitude=slice(0, 1248),  # (0., 125.) deg
    longitude=slice(0, 1248),  # (90., -35.) deg
)

In [10]:
#Making coordinates consistent
NIRv = NIRv.assign_coords(
    longitude=NIRv.longitude.round(4),
    latitude=NIRv.latitude.round(4)
)
SSM = SSM.assign_coords(
    longitude=NIRv.longitude.round(4),
    latitude=NIRv.latitude.round(4)
)
TWS = TWS.assign_coords(
    longitude=NIRv.longitude.round(4),
    latitude=NIRv.latitude.round(4)
)
Ta = Ta.assign_coords(
    longitude=NIRv.longitude.round(4),
    latitude=NIRv.latitude.round(4)
)
Rn = Rn.assign_coords(
    longitude=NIRv.longitude.round(4),
    latitude=NIRv.latitude.round(4)
)

In [11]:
# List of datasets
datasets = [NIRv, SSM, TWS, Ta, Rn]

In [12]:
# Manually define chunk sizes for latitude and longitude
latitude_chunks = 1248 // 4  # Divide by the number of workers
longitude_chunks = 1248 // 4  # Divide by the number of workers

In [13]:
# Chunk and stack datasets lazily
chunked_datasets = [ds.chunk({'latitude': latitude_chunks, 'longitude': longitude_chunks, 'time': -1}) for ds in datasets]

In [14]:
# STacking the chunked_datasets
stacked_dataset = xr.concat(chunked_datasets, dim='variable_name')

In [15]:
#Defining INPUT and output variables 
INPUT_VARIABLES = [
 'NIRv','SSM','TWS','Ta','Rn']

OUTPUT_VARIABLES = ['n','r','CI95_lower','CI95_upper','p_value']

In [16]:
#getting chunk size
chunks = [stacked_dataset.chunksizes[v] for v in ['time', 'latitude', 'longitude']]

In [17]:
#adding output variables dimension
chunks.append((len(OUTPUT_VARIABLES),))

In [32]:
#Since I compute partial correlation, i do not need time information but different layers for each output.
template_pc = xr.DataArray(
    name = 'PC',
    data=da.zeros(
        (len(stacked_dataset.latitude), len(stacked_dataset.longitude), len(OUTPUT_VARIABLES)), 
        chunks=chunks[1:4],
    ),
    dims=("latitude", "longitude", "output_variable"),
    coords={
        "output_variable": OUTPUT_VARIABLES, 
        "latitude": stacked_dataset.latitude,
        "longitude": stacked_dataset.longitude
    }
)

In [41]:
#I define the function which takes dask arrays and compute partial correlation from penguoin package.  
def partial_correlation(ds):
    #Grabbing value
    NIRv = ds['NIRv'].data.reshape(1, -1)
    SSM = ds['SSM'].data.reshape(1, -1)
    TWS = ds['TWS'].data.reshape(1, -1)
    Ta = ds['Ta'].data.reshape(1, -1)
    Rn = ds['Rn'].data.reshape(1, -1)
 
    #Concatenate in columns
    features_arr = np.concatenate((
        NIRv, SSM, TWS, Ta, Rn
    ),axis=1)

    #Creating pandas dataframe and dropping NA value
    df_features = pd.DataFrame(
    data=features_arr,
    columns=INPUT_VARIABLES,
    ).dropna
    
    # CCalculating partial correlation if there are more than 40 rows else returning NAN. 
    if df_features.shape[0] >=40:
        partial_corr_result = pg.partial_corr(df_features, x='NIRv', y='SSM', covar=['TWS', 'Ta', 'Rn'], method='spearman').round(4)
        PC = np.array([partial_corr_result['n'].iloc[0],partial_corr_result['r'].iloc[0],partial_corr_result['CI95%'].iloc[0][0],partial_corr_result['CI95%'].iloc[0][1],partial_corr_result['p-val'].iloc[0]]) #Grabbing results in numpy array
    else:
        PC = np.array([np.nan, np.nan, np.nan, np.nan, np.nan])
    return xr.DataArray(
        name='PC',
        data=PC.reshape(len(ds.latitude), len(ds.longitude), len(OUTPUT_VARIABLES)),
        dims=( "latitude", "longitude", "output_variable"),
        coords={
            "output_variable": OUTPUT_VARIABLES, 
            "latitude": ds.latitude,
            "longitude":ds.longitude
        }
    )


In [42]:
PC = xr.map_blocks(
    partial_correlation,
    stacked_dataset,
    template=template_pc,
)

In [43]:
PC.compute()

2024-01-17 20:10:03,065 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/distributed/protocol/core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/msgpack/fallback.py", line 136, in unpackb
    raise ExtraData(ret, unpacker._get_extradata())
msgpack.exceptions.ExtraData: unpack(b) received extra data.
2024-01-17 20:10:03,081 - distributed.core - ERROR - Exception while handling op register-client
Traceback (most recent call last):
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/distributed/core.py", line 968, in _handle_comm
    result = await result
             ^^^^^^^^^^^^
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/distributed/scheduler.py", line 5532, in add_client
    await self.handle_stream(comm=comm, extra={"client": client})
  File "/home/khanalp/anac

CancelledError: ('<this-array>-partial_correlation-88e9a3518916466fe37395f36dfec150', 0, 0, 0)

2024-01-17 19:55:00,811 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/distributed/protocol/core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/msgpack/fallback.py", line 136, in unpackb
    raise ExtraData(ret, unpacker._get_extradata())
msgpack.exceptions.ExtraData: unpack(b) received extra data.
2024-01-17 19:55:00,817 - distributed.core - ERROR - Exception while handling op register-client
Traceback (most recent call last):
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/distributed/core.py", line 968, in _handle_comm
    result = await result
             ^^^^^^^^^^^^
  File "/home/khanalp/anaconda3/lib/python3.11/site-packages/distributed/scheduler.py", line 5532, in add_client
    await self.handle_stream(comm=comm, extra={"client": client})
  File "/home/khanalp/anac

CancelledError: ('getitem-dacbc6e4f4f3d84c14fec8062327ca20', 2, 0)